In [219]:
import pandas as pd
import numpy as np
import selenium
from helium import *
from bs4 import BeautifulSoup as bsp
import requests
import time
from tqdm import tqdm
import math
import re

In [168]:
def extract_word(text):
    hangul = re.compile('[^가-힣 a-zA-Z]') 
    result = hangul.sub('', text) 
    return result

In [220]:
def youtube_crawl(url):
    driver=start_chrome()
    go_to(url)
    time.sleep(10)
    # press(SPACE)
    
    count=0
    cnt=find_all(S('.count-text.style-scope.ytd-comments-header-renderer > .style-scope.yt-formatted-string'))[1].web_element.text
    cnt=cnt.replace(',','')
    cnt=int(cnt)
    final=math.ceil((cnt/20)*1.5)
    while count<final:
        scroll_down(num_pixels=1000000)
        time.sleep(0.5)
        count+=1
    time.sleep(2)
    try:
        wait_until(lambda: click(S('.more-button.style-scope.ytd-comment-renderer')),timeout_secs=2000, interval_secs=1)
    except:
        pass
    time.sleep(2)

    html_source = driver.page_source
    soup = bsp(html_source, 'lxml')
    comment=soup.select('yt-formatted-string#content-text')
    youtube={}
    youtube['comment']=[]
    for i in range(len(comment)):
        text=str(comment[i].text)
        youtube['comment'].append(text)
    title=find_all(S('#title > .style-scope.ytd-watch-metadata'))[-1].web_element.text
    kill_browser()
    youtube=pd.DataFrame.from_dict(youtube)
    youtube['comment']=youtube['comment'].apply(lambda x: extract_word(x))
    print("crawling_OK")
    return youtube.to_csv(f'./{title}.csv',index=False)

In [221]:
youtube_crawl('https://www.youtube.com/watch?v=229AALYeH90')

crawling_OK
